In [6]:
from dash import Dash, dcc, html, dash_table
import pandas as pd
import plotly.express as px
import dash_leaflet as dl
from dash.dependencies import Input, Output
import base64
import os

# Load data from CSV
data_file = "aac_shelter_outcomes.csv"
df = pd.read_csv(data_file)

# Ensure correct data types
df["age_upon_outcome_in_weeks"] = pd.to_numeric(df["age_upon_outcome_in_weeks"], errors='coerce')
df["location_lat"] = pd.to_numeric(df["location_lat"], errors='coerce')
df["location_long"] = pd.to_numeric(df["location_long"], errors='coerce')

## print("Loaded data preview:")
## print(df.head())  # Debugging

def fetch_data(filter_option):
    """Fetch and filter data based on rescue type."""
    if filter_option == "water":
        return df[df["breed"].str.contains("lab|chesa|newf", case=False, na=False) & (df["age_upon_outcome_in_weeks"] >= 26)]
    elif filter_option == "mountain":
        return df[df["breed"].str.contains("german|mala|old english sheep|husk|rott", case=False, na=False) & (df["age_upon_outcome_in_weeks"] >= 26)]
    elif filter_option == "disaster":
        return df[df["breed"].str.contains("german|golden|blood|dober|rott", case=False, na=False) & (df["age_upon_outcome_in_weeks"].between(20, 300))]
    return df  # Reset returns all data

df_filtered = fetch_data("reset")

app = Dash(__name__)

image_filename = 'Grazioso Salvare Logo.png' 
if os.path.exists(image_filename):
    encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()
    logo_img = html.Img(src=f'data:image/png;base64,{encoded_image}', style={'width': '200px'})
else:
    logo_img = None

# Layout
app.layout = html.Div([
    html.Center(html.A(href="https://www.snhu.edu", children=logo_img)),
    html.Center(html.B(html.H1("Reese's CS-340 Dashboard"))),
    html.Hr(),
    dcc.RadioItems(
        id='filter-options',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain Rescue', 'value': 'mountain'},
            {'label': 'Disaster Tracking', 'value': 'disaster'},
            {'label': 'Reset', 'value': 'reset'}
        ],
        value='reset'
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df_filtered.to_dict('records'),
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        selected_rows=[0],  # Auto-select the first row
        page_action="native",
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    html.Div([
        html.Div(id='graph-id', className='six columns'),
        html.Div(id='map-id', className='six columns')
    ], className='row')
])

@app.callback(
    [Output('datatable-id', 'data'), Output('datatable-id', 'selected_rows')],
    [Input('filter-options', 'value')]
)
def update_table(filter_option):
    data = fetch_data(filter_option)
    print(f"Updated table with filter: {filter_option}, rows: {len(data)}")
    return data.to_dict('records'), [0]  # Auto-select first row

@app.callback(Output('graph-id', "children"), [Input('datatable-id', "derived_virtual_data")])
def update_graph(viewData):
    dff = pd.DataFrame(viewData) if viewData else pd.DataFrame()
    if dff.empty:
        return html.P("No data available.")
    return dcc.Graph(figure=px.pie(dff, names='breed'))

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"), Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, selected_rows):
    if not viewData or not selected_rows:
        print("Map update: No data or no row selected.")
        return dl.Map(center=[30.75, -97.48], zoom=10, children=[dl.TileLayer()])

    dff = pd.DataFrame(viewData)
    if dff.empty or 'location_lat' not in dff.columns or 'location_long' not in dff.columns:
        print("Map update: DataFrame empty or missing lat/long.")
        return html.P("Location data missing.")

    ## print("Map update: Data received:", dff[['location_lat', 'location_long']].head())

    row = selected_rows[0] if selected_rows else 0  # Default to first row
    lat = dff.loc[row, 'location_lat'] if not pd.isna(dff.loc[row, 'location_lat']) else 30.75
    long = dff.loc[row, 'location_long'] if not pd.isna(dff.loc[row, 'location_long']) else -97.48
    tooltip = dff.loc[row, 'name'] if 'name' in dff.columns else "Unknown"

    ## print(f"Map update: Centering on ({lat}, {long}) with tooltip {tooltip}")

    return dl.Map(
        style={'width': '1000px', 'height': '500px'},
        center=[lat, long], zoom=10,
        children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(
                position=[lat, long],
                children=[dl.Tooltip(tooltip)]
            )
        ]
    )

if __name__ == '__main__':
    app.run(debug=True, port=8051)


Updated table with filter: reset, rows: 10000
Map update: No data or no row selected.
Updated table with filter: reset, rows: 10000
Map update: No data or no row selected.
Updated table with filter: disaster, rows: 351
Updated table with filter: mountain, rows: 440
Updated table with filter: disaster, rows: 351


127.0.0.1 - - [30/Mar/2025 12:09:33] code 400, message Bad request version ('\x00\x00\x00Windows_NT\x00\x00\x00\x04compression\x00\x11\x00\x00\x00\x020\x00\x05\x00\x00\x00none\x00\x00\x00')


Updated table with filter: reset, rows: 10000Map update: No data or no row selected.

